In [1]:
!pip install sparqlwrapper
!pip install pandas
%pip install aiohttp aiofiles nest_asyncio

   ---------------------------------------- 0.0/363.4 kB ? eta -:--:--
   ---------- ----------------------------- 92.2/363.4 kB 2.6 MB/s eta 0:00:01
   --------------------------------- ------ 307.2/363.4 kB 3.2 MB/s eta 0:00:01
   ---------------------------------------- 363.4/363.4 kB 3.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/50.5 kB ? eta -:--:--
   ---------------------------------------- 50.5/50.5 kB ? eta 0:00:00
   ---------------------------------------- 0.0/76.4 kB ? eta -:--:--
   ---------------------------------------- 76.4/76.4 kB 4.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
try:
    os.mkdir('images')
except:
    pass

In [ ]:
import sys
from SPARQLWrapper import SPARQLWrapper, JSON
import requests
import shutil
import os
import pandas as pd

endpoint_url = "https://query.wikidata.org/sparql"

# Get cities
query = """SELECT ?image WHERE {
  ?item wdt:P31 wd:Q8502 . # Check if item is a mountain
  ?item wdt:P18 ?image .
}
LIMIT 150
"""


def get_results(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s" % (
        sys.version_info[0],
        sys.version_info[1],
    )
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()

def download_image(url):

    headers = {"User-Agent": "Mozilla/5.0"}
    request = requests.get(url, allow_redirects=True, headers=headers, stream=True)
    if request.status_code == 200:
        nom = os.path.basename(url)[-35:]
        chemin = "images/"+nom
        with open(chemin, "wb") as image:
            request.raw.decode_content = True
            shutil.copyfileobj(request.raw, image)
    return request.status_code

array = []
results = get_results(endpoint_url, query)

for result in results["results"]["bindings"]:
    array.append(
        (
            result["image"]["value"],
        )
    )
    
dataframe = pd.DataFrame(array, columns=["image"])
dataframe = dataframe.astype(
    dtype={"image": "<U200"}
)
dataframe.image.apply(download_image)

dataframe

In [19]:
import os
import json
from PIL import Image, ExifTags, TiffImagePlugin

def extract_metadata(image_path):
  """
  Extracts metadata from a single image.

  Args:
      image_path: Path to the image file.

  Returns:
      A dictionary containing the extracted metadata.
  """
  try:
    image = Image.open(image_path)
    exif_data = image._getexif()
    metadata = dict()
    for key, value in exif_data.items():
      if isinstance(value, bytes):
        continue
      if (not isinstance(value, int)) and (not isinstance(value, float)):
        value = f"{value}"
      if key in ExifTags.TAGS:
        metadata[ExifTags.TAGS[key]] = value
    return metadata
  except:
    print(f"Error processing image: {image_path}")
    return {}


"""
Extracts metadata from all images in a folder and saves it to a JSON file.
"""
# Modify this path to your desired directory containing images
image_dir = "./images"

all_metadata = []
for filename in os.listdir(image_dir):
  if filename.lower().endswith((".jpg", ".jpeg", ".png")):
    image_path = os.path.join(image_dir, filename)
    metadata = extract_metadata(image_path)
    all_metadata.append(metadata)

# Modify this filename to your desired output file name
output_file = "image_metadata.json"

with open(output_file, "w") as f:
  json.dump(all_metadata, f, indent=4)

print(f"Image metadata saved to: {output_file}")

Error processing image: ./images\%20Crater%20Lake%20%28052005%29.jpg
Error processing image: ./images\%20geograph.org.uk%20-%20958044.jpg
Error processing image: ./images\0de%20la%20Barre%20des%20Ecrins.png
Error processing image: ./images\0do%20Monte%20Roraima%20%281%29.jpg
Error processing image: ./images\20Gabelhorn%20und%20Wellenkuppe.jpg
Error processing image: ./images\20geograph.org.uk%20-%201247806.jpg
Error processing image: ./images\20geograph.org.uk%20-%201456731.jpg
Error processing image: ./images\20geograph.org.uk%20-%201457518.jpg
Error processing image: ./images\20geograph.org.uk%20-%201657454.jpg
Error processing image: ./images\31MonteGeneroso.jpg
Error processing image: ./images\AiguilleduGeant0001.jpg
Error processing image: ./images\AiguillesPeuterey0002.jpg
Error processing image: ./images\Allalinhorn0001.jpg
Error processing image: ./images\Bos%20Fulen.jpg
Error processing image: ./images\Bur%20ni%20Telong.jpg
Error processing image: ./images\Drus-verte.jpg
Error

In [5]:
import sys
from SPARQLWrapper import SPARQLWrapper, JSON
import requests
import shutil
import os
import pandas as pd
import nest_asyncio
import asyncio
import aiofiles
import aiohttp
from pandas import DataFrame

endpoint_url = "https://query.wikidata.org/sparql"

# Get mountains
query = """
    SELECT ?item ?itemLabel ?part ?partLabel ?image WHERE {
      ?item wdt:P31 wd:Q8502 .
      ?item wdt:P361 ?part .
      ?item wdt:P18 ?image .

    SERVICE wikibase:label { #BabelRainbow
        bd:serviceParam wikibase:language "[AUTO_LANGUAGE],fr"
    }

    }
    LIMIT 100
"""
nest_asyncio.apply()

download_sem = asyncio.Semaphore(4)

async def download_images(df: DataFrame):
    download_tasks = []
    print(f"Téléchargement de {len(df)} images...")
    for _, row in df.iterrows():
        url = row["image"]
        download_tasks.append(download_image(url))
    await asyncio.gather(*download_tasks)

def get_results(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s" % (
        sys.version_info[0],
        sys.version_info[1],
    )
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()

async def download_image(url):
    headers = {"User-Agent": "Mozilla/5.0"}
    async with aiohttp.ClientSession() as session:
            async with download_sem:
                async with session.get(url, allow_redirects=True, headers=headers) as res:
                    if res.status != 200:
                        print(f"Impossible de télécharger l'image à l'url {url} (code {res.status})")
                        return
                    nom = os.path.basename(url)[-35:]
                    chemin = "images/"+nom
                    async with aiofiles.open(f"{chemin}", "wb") as f:
                        await f.write(await res.read())
                        await f.close()

array = []
results = get_results(endpoint_url, query)

for result in results["results"]["bindings"]:
    element = []
    element.append(result["item"]["value"])
    element.append(result["itemLabel"]["value"])
    element.append(result["image"]["value"])
    element.append(result["partLabel"]["value"])
    array.append(element)
    
dataframe = pd.DataFrame(array, columns=["qid", "itemLabel", "image", "partLabel"])


dataframe

,qid,itemLabel,image,partLabel
0,http://www.wikidata.org/entity/Q513,Everest,http://commons.wikimedia.org/wiki/Special:File...,Himalaya
1,http://www.wikidata.org/entity/Q513,Everest,http://commons.wikimedia.org/wiki/Special:File...,sommets de plus de huit mille mètres
2,http://www.wikidata.org/entity/Q513,Everest,http://commons.wikimedia.org/wiki/Special:File...,sept sommets
3,http://www.wikidata.org/entity/Q513,Everest,http://commons.wikimedia.org/wiki/Special:File...,sommet ultra-proéminent
4,http://www.wikidata.org/entity/Q583,mont Blanc,http://commons.wikimedia.org/wiki/Special:File...,sept sommets
...,...,...,...,...
95,http://www.wikidata.org/entity/Q122225,Seongsan Ilchulbong,http://commons.wikimedia.org/wiki/Special:File...,île volcanique et tunnels de lave de Jeju
96,http://www.wikidata.org/entity/Q122225,Seongsan Ilchulbong,http://commons.wikimedia.org/wiki/Special:File...,île volcanique et tunnels de lave de Jeju
97,http://www.wikidata.org/entity/Q125156,Alutu,http://commons.wikimedia.org/wiki/Special:File...,vallée du grand rift
98,http://www.wikidata.org/entity/Q542041,mont Townsend,http://commons.wikimedia.org/wiki/Special:File...,Sept seconds sommets


In [6]:
asyncio.run(download_images(dataframe))

Téléchargement de 100 images...
